# Finding and using anchor points

In this notebook, we show how to find anchor points based on your training set and how to use them to estimate the performance of new models in the test set.

## Preparing data

Loading packages

In [32]:
import numpy as np
import pickle
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import pairwise_distances
from irt import *
from utils import *

random_state = 42

The leaderboard dataset we will use is composed by six scenarios (sub-datasets):
1. TruthfulQA
1. GSM8K
1. Winogrande
1. ARC
1. HellaSwag
1. MMLU

MMLU is further divided into sub-scenarios (e.g., abstract algebra, anatomy, etc). Let's check scenarios and sub-scenarios:

In [33]:
scenarios
scenarios = {'wmdp_cyber':['wmdp_cyber'],
             'wmdp_bio':['wmdp_bio'],}

Loading leaderboard data:

In [34]:
with open('wmdp_data.pkl', 'rb') as handle:
    data = pickle.load(handle)

In this dataset, we have data from 395 models. Let's see the names of some of them below

In [35]:
len(data['models']),data['models'][:10]

(64,
 ['microsoft/Phi-3-medium-4k-instruct',
  'internlm/internlm2_5-7b-chat',
  '01-ai/Yi-1.5-9B-Chat',
  'MaziyarPanahi/Llama-3-8B-Instruct-v0.8',
  'Qwen/Qwen2-7B-Instruct',
  'NousResearch/Hermes-2-Theta-Llama-3-8B',
  'vicgalle/Roleplay-Llama-3-8B',
  'Qwen/Qwen2-7B',
  'NousResearch/Nous-Hermes-2-SOLAR-10.7B',
  'UCLA-AGI/Llama-3-Instruct-8B-SPPO-Iter3'])

Below, we will process the data so all correctness scores (for all scenarios) are stored in $Y$. The dictionaries `scenarios_position` and `subscenarios_position` give the position of scenarios/subscenarios correctness scores in $Y$.

In [36]:
scenarios_position, subscenarios_position = prepare_data(scenarios, data)
Y = create_responses(scenarios, data)
Y.shape

(64, 3260)

For example, below you can see the scores for MMLU:

For scenarios that have multiple subscenarios, it is usually the case that we want to give equal importance to individual subscenarios when computing the aggregated performance in that scenario. This is equivalent to using a weighted average when computing the aggregated performance. We will create `balance_weights`, a vector of weights to help us compute those weighted averages. These weights will be different than one only for MMLU, which is the only scenario with multiple subscenarios.

In [37]:
balance_weights = np.ones(Y.shape[1])

# N = len(scenarios_position['mmlu'])
# n_sub = len(scenarios['mmlu'])
# for sub in scenarios['mmlu']:
#     n_i = len(subscenarios_position['mmlu'][sub])
#     balance_weights[subscenarios_position['mmlu'][sub]] = N/(n_sub*n_i)  

We can see below that first averaging within subscenarios and then computing a simple average is equivalent to using a weighted average from the beginning:

In [8]:
# accs1 = np.mean([Y[:,subscenarios_position['mmlu'][sub]].mean(axis=1) for sub in scenarios['mmlu']], axis=0)
# accs2 = (balance_weights*Y)[:,scenarios_position['mmlu']].mean(axis=1)

# np.abs(accs1 - accs2).mean()

2.322333605307685e-14

## Getting and using anchor points

Let's split the data in train and test (recent models are placed in the test set):

In [38]:
Y_test = Y[:16]
Y_train = Y[16:]

The variable `number_item` gives the number of anchor points we want to find in each scenario:

In [55]:
number_item = 64

The variable `clustering` specified how the clusting is run. If `clustering="correct."`, then correctness is used. On the other hand, if `clustering="irt"`, then the IRT embeddings for examples are used.

In [56]:
clustering = 'irt' # 'correct.' or 'irt'

Computing anchor points and their weights for each scenario:

In [57]:
anchor_points = {}
anchor_weights = {}

for scenario in scenarios.keys():

    if clustering=='correct.':
        X = Y_train[:,scenarios_position[scenario]].T
    elif clustering=='irt':
        A, B, _ = load_irt_parameters('data/irt_model/')
        X = np.vstack((A.squeeze(), B.squeeze().reshape((1,-1)))).T
        X = X[scenarios_position[scenario]]
    else:
        raise NotImplementedError 
        
    #Normalizing balance_weights, so their sum is one within each scenario
    norm_balance_weights = balance_weights[scenarios_position[scenario]]
    norm_balance_weights /= norm_balance_weights.sum()

    # Fitting the KMeans model
    kmeans = KMeans(n_clusters=number_item, n_init="auto", random_state=random_state)
    kmeans.fit(X, sample_weight=norm_balance_weights)

    # Calculating anchor points
    anchor_points[scenario] = pairwise_distances(kmeans.cluster_centers_, X, metric='euclidean').argmin(axis=1)

    # Calculating anchor weights
    anchor_weights[scenario] = np.array([np.sum(norm_balance_weights[kmeans.labels_==c]) for c in range(number_item)])

Saving

In [58]:
anchor = {'anchor_points':anchor_points,
          'anchor_weights':anchor_weights}

with open('data/anchor.pickle', 'wb') as handle:
    pickle.dump(anchor, handle, protocol=pickle.HIGHEST_PROTOCOL)

Checking results

In [59]:
anchor_points['wmdp_cyber']

array([1126,  201,  136, 1851, 1159, 1175, 1630,  425,  777, 1710, 1661,
       1129,  473,  399, 1442, 1312, 1689, 1745,  696, 1101,  831, 1494,
        683,  480,  500,   65, 1097, 1341,  300, 1400,  330,  454, 1950,
       1751, 1761,  591,  345,  254, 1003, 1037,  999,  735, 1463,  750,
        987,  638,   11,  353, 1540,   85, 1533, 1868, 1420,  126,  716,
       1680,  191,  563,  403, 1046, 1802, 1235, 1546,  682,  600,  827,
       1142, 1545, 1176,  782, 1106,  261, 1396, 1554, 1819,  451, 1252,
          7, 1052, 1030, 1864,  429,  173,  780, 1196, 1156, 1757,  784,
       1862,  954,  408,  669,  601, 1916, 1604, 1443,  986, 1529, 1361,
        263])

In [60]:
anchor_weights['wmdp_cyber']

array([0.01207851, 0.01258178, 0.0140916 , 0.00905888, 0.01157524,
       0.00603926, 0.02365375, 0.00905888, 0.00754907, 0.01258178,
       0.00553598, 0.00503271, 0.00905888, 0.0140916 , 0.00855561,
       0.0105687 , 0.00855561, 0.00805234, 0.01610468, 0.00855561,
       0.01358832, 0.01157524, 0.00654253, 0.00855561, 0.00855561,
       0.00855561, 0.00754907, 0.00452944, 0.00855561, 0.01006543,
       0.0035229 , 0.0070458 , 0.00754907, 0.00805234, 0.01006543,
       0.01711122, 0.01509814, 0.00956215, 0.0070458 , 0.01258178,
       0.01107197, 0.00603926, 0.02013085, 0.0105687 , 0.01207851,
       0.00603926, 0.01006543, 0.01006543, 0.02717665, 0.00654253,
       0.00905888, 0.00855561, 0.05485657, 0.01157524, 0.00402617,
       0.00805234, 0.01509814, 0.00855561, 0.0140916 , 0.00654253,
       0.00855561, 0.00956215, 0.00805234, 0.00905888, 0.01207851,
       0.00805234, 0.00503271, 0.00754907, 0.00452944, 0.00553598,
       0.00503271, 0.00402617, 0.00603926, 0.00805234, 0.01006

Using anchor points to estimate performance in the test set and reporting the average prediction error

In [61]:
for scenario in scenarios.keys():
    Y_anchor = Y_test[:,scenarios_position[scenario]][:,anchor_points[scenario]]
    Y_hat = (Y_anchor*anchor_weights[scenario]).sum(axis=1)
    Y_true = (balance_weights*Y_test)[:,scenarios_position[scenario]].mean(axis=1)
    avg_error = np.abs(Y_hat-Y_true).mean()
    sd = np.abs(Y_hat-Y_true).std()

    print(f"scenario: {scenario}, avg. error: {avg_error:.3f}, std. dev.: {sd:.3f}")

scenario: wmdp_cyber, avg. error: 0.024, std. dev.: 0.015
scenario: wmdp_bio, avg. error: 0.024, std. dev.: 0.018
